In [5]:
import tensorflow as tf
import os
import cv2
import numpy as np
from tensorflow.keras import layers, optimizers, losses, metrics, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

In [2]:
# Main directory containing test images
test_main_directory = r"C:\Users\91934\Music\UROP\dataset\sub_test"  # Replace with the actual path

# Number of classes
num_classes = 5  # Update with the actual number of classes

# Initialize lists to store test images and labels
test_images = []
test_labels = []

# Iterate through test directories
for class_label in range(num_classes):
    # Directory for the current class in the test set
    test_class_directory = os.path.join(test_main_directory, f"{class_label}")

    # Iterate through image files in the test directory for the current class
    for image_file in os.listdir(test_class_directory):
        # Load and preprocess the image using OpenCV
        image_path = os.path.join(test_class_directory, image_file)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB if needed

        # Optional: Resize the image to match the input size expected by ResNet50
        image = cv2.resize(image, (224, 224))

        # Append the image and label to the lists
        test_images.append(image)
        test_labels.append(class_label)

# Convert the lists to NumPy arrays
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Shuffle the test data
random_indices_test = np.random.permutation(len(test_images))
test_images = test_images[random_indices_test]
test_labels = test_labels[random_indices_test]

# Print the shuffled test data
print("Shuffled Test Images:", test_images)
print("Shuffled Test Labels:", test_labels)

Shuffled Test Images: [[[[3 1 2]
   [3 1 2]
   [3 1 2]
   ...
   [3 1 2]
   [3 1 2]
   [3 1 2]]

  [[3 1 2]
   [3 1 2]
   [3 1 2]
   ...
   [3 1 2]
   [3 1 2]
   [3 1 2]]

  [[3 1 2]
   [3 1 2]
   [3 1 2]
   ...
   [3 1 2]
   [3 1 2]
   [3 1 2]]

  ...

  [[2 2 2]
   [0 0 0]
   [2 2 2]
   ...
   [2 2 2]
   [2 2 2]
   [2 2 2]]

  [[2 2 2]
   [1 1 1]
   [1 1 1]
   ...
   [2 2 2]
   [2 2 2]
   [2 2 2]]

  [[2 2 2]
   [2 2 2]
   [1 1 1]
   ...
   [2 2 2]
   [2 2 2]
   [2 2 2]]]


 [[[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]

  ...

  [[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]]


 [[[3 2 0]
   [2 1 0]
   [2 1 0]
   ...
   [1 1 1]
   [1 1 1

In [3]:
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    image_array = img_to_array(image)
    image_array /= 255.0

    return image_array

In [4]:
main_directory = r"C:\Users\91934\Music\UROP\dataset\sub_train"  
train_images = []
train_labels = []
num_classes = 10
# Iterate through each subdirectory
for class_label in range(num_classes):  
    # Directory for the current class
    class_directory = os.path.join(main_directory, f"td{class_label}")

    # Iterate through image files in the class directory
    for image_file in os.listdir(class_directory):
        # Construct the path to the image file
        image_path = os.path.join(class_directory, image_file)

        # Load and preprocess the image
        image_array = load_and_preprocess_image(image_path)

        # Append the image and label to the lists
        train_images.append(image_array)
        train_labels.append(class_label)

# Convert the lists to NumPy arrays
train_images = np.array(train_images)
train_labels = to_categorical(np.array(train_labels)) 

In [5]:
validation_main_directory = r"C:\Users\91934\Music\UROP\dataset\sub_valid"
valid_images = []
valid_labels = []
num_classes = 5
for class_label in range(num_classes):
    # Directory for the current class in the test set
    valid_class_directory = os.path.join(validation_main_directory, f"td{class_label}")
    # Iterate through image files in the test directory for the current class
    for image_file in os.listdir(valid_class_directory):
        # Load and preprocess the image
        image_path = os.path.join(valid_class_directory, image_file)
        image_array = load_and_preprocess_image(image_path)

        # Append the image and label to the lists
        valid_images.append(image_array)
        valid_labels.append(class_label)

# Convert the lists to NumPy arrays
valid_images = np.array(valid_images)
valid_labels = to_categorical(np.array(valid_labels))


In [6]:
from tensorflow.keras.optimizers import Adam

def create_resnet50_model(input_shape,num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs = base_model.input, outputs = predictions)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss=tf.losses.CategoricalCrossentropy(), metrics=['accuracy'])
    return model


In [7]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

def scheduler_function(epoch, learning_rate):
    if epoch < 5:
        return learning_rate
    else:
        return learning_rate * tf.math.exp(-0.1)

learning_rate_scheduler = LearningRateScheduler(scheduler_function)

In [8]:
input_shape = (224, 224, 3)
num_classes = 5
model = create_resnet50_model(input_shape, num_classes)

# Train the model
model.fit(train_images, train_labels, epochs=1,batch_size=32, validation_data=(valid_images, valid_labels), callbacks=[early_stopping,learning_rate_scheduler])

# Save the model weights 
model.save_weights('resnet50_model_weights.h5')

92/92 [==============================] - 870s 9s/step - loss: 0.6656 - accuracy: 0.7570 - val_loss: 21.2453 - val_accuracy: 0.4699 - lr: 1.0000e-04


In [10]:
input_shape = (224, 224, 3)
num_classes = 5
model = create_resnet50_model(input_shape, num_classes)

# Train the model
model.fit(train_images, train_labels, epochs=5,batch_size=32, validation_data=(valid_images, valid_labels), callbacks=[early_stopping,learning_rate_scheduler])


Epoch 1/10
92/92 [==============================] - 908s 10s/step - loss: 0.6616 - accuracy: 0.7597 - val_loss: 15.9496 - val_accuracy: 0.4699 - lr: 1.0000e-04
Epoch 2/10
92/92 [==============================] - 904s 10s/step - loss: 0.3329 - accuracy: 0.8805 - val_loss: 10.5979 - val_accuracy: 0.4699 - lr: 1.0000e-04
Epoch 3/10
92/92 [==============================] - 915s 10s/step - loss: 0.1776 - accuracy: 0.9386 - val_loss: 8.4061 - val_accuracy: 0.4699 - lr: 1.0000e-04
Epoch 4/10
92/92 [==============================] - 914s 10s/step - loss: 0.1065 - accuracy: 0.9669 - val_loss: 9.4805 - val_accuracy: 0.4699 - lr: 1.0000e-04
Epoch 5/10
92/92 [==============================] - 917s 10s/step - loss: 0.0693 - accuracy: 0.9761 - val_loss: 10.3580 - val_accuracy: 0.4699 - lr: 1.0000e-04


In [11]:
model.save_weights('resnet50_model_weights.h5')

In [12]:
new_model = create_resnet50_model(input_shape, num_classes)

In [14]:
for class_label in range(num_classes):
    aggregated_weights = [tf.zeros_like(w) for w in model.get_weights()]

    # Load the saved model weights
    model.load_weights('resnet50_model_weights.h5')

    # Directory for the current class
    class_directory = os.path.join(main_directory, f"{class_label}")

    # Iterate through image files in the class directory
    for image_file in os.listdir(class_directory):
        # Construct the path to the image file
        image_path = os.path.join(class_directory, image_file)

        # Load and preprocess the image
        image_array = load_and_preprocess_image(image_path)

        # Aggregate the model weights
        model_predictions = model.predict(np.array([image_array]))
        model_weights = model.get_weights()
        aggregated_weights = [aggregated + w for aggregated, w in zip(aggregated_weights, model_weights)]
    aggregated_weights = [aggregated / num_classes for aggregated in aggregated_weights]
    new_model.set_weights(aggregated_weights )

new_model.save_weights('averaged_resnet50_model_weights.h5')

1/1 [==============================] - 1s 554ms/step


In [27]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

# Load and preprocess the test images
test_main_directory = r"C:\Users\91934\Music\UROP\dataset\sub_test"
test_images = []
test_labels = []
num_classes = 5
for class_label in range(num_classes):
    # Directory for the current class in the test set
    test_class_directory = os.path.join(test_main_directory, f"td{class_label}")

    # Iterate through image files in the test directory for the current class
    for image_file in os.listdir(test_class_directory):
        # Load and preprocess the image
        image_path = os.path.join(test_class_directory, image_file)
        image_array = load_and_preprocess_image(image_path)

        # Append the image and label to the lists
        test_images.append(image_array)
        test_labels.append(class_label)

# Convert the lists to NumPy arrays
test_subset_images = np.array(test_images)
test_subset_labels = to_categorical(np.array(test_labels)) 

# Test the accuracy of the new model
test_loss, test_accuracy = new_model.evaluate(test_subset_images, test_subset_labels)
print("Aggregated model accuracy:", test_accuracy)

7/7 [==============================] - 32s 5s/step - loss: nan - accuracy: 0.9045
Aggregated model accuracy: 0.9045454263687134
